In [14]:
import numpy as np
import h5py
from tqdm.auto import tqdm

In [57]:
test_file = h5py.File('data_M1.h5py', 'r')
i_dict = {}
for k in tqdm(test_file.keys()):
    for chunk in test_file[k].iter_chunks():
        #print(chunk)
        #continue
        if np.sum(test_file[k][chunk][-1]) == 0:
            for i, v in enumerate(test_file[k][chunk]):
                #print(v)
                if np.sum(v) == 0:
                    i_dict[k] = chunk[0].start+i
                    break
            break

  0%|          | 0/1024 [00:00<?, ?it/s]

In [62]:
all_file = h5py.File('data_all.h5py', 'a')
for k in tqdm(test_file.keys()):
    if k not in all_file:
        all_file.create_dataset(
            k,
            (1_000_000, 768),
            compression="lzf",
            dtype='int8',
            chunks=(10_000,768),
            maxshape=(None, 768),
        )
        for chunk in test_file.iter_chunks():
            if chunk[0] < i_dict[k]:
                all_file[k][chunk] = test_file[k][chunk]
            else:
                for i, v in enumerate(test_file[k][chunk]):
                    while np.sum(v) != 0:
                        i_dict[k] = chunk[0].start+i

7584861

In [69]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore
	modified:   create_hdf5.py
	modified:   preprocessing_notebook.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Untitled.ipynb
	check_cuda.py
	check_h5py.ipynb
	create_all.sh
	create_vector_script.py
	last_index_100px.txt
	preprocess.ipynb
	preprocessing.py
	untitled.txt
	visualize_lsh.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [5]:
for chunk_size in (np.arange(20)+1)*10_000:
    try:
        file.close()
    except:
        pass
    file = h5py.File('chunktest.h5py','w')
    file.create_dataset(
        'test',
        (1_000_000, 768),
        compression="lzf",
        dtype='int8',
        chunks=(chunk_size,768),
        maxshape=(None, 768),
    )
    np.random.seed(42)
    file['test'][:] = np.random.randint(low=-127, high=127, size=(1_000_000, 768), dtype=np.int8)
    buckets = []
    vecs = []
    times = []
    with h5py.File('chunktest.h5py','r') as file:
        for key in file.keys():
            print(len(file[key]))

            for c in file[key].iter_chunks():
                start = time.time()
                np.array(file[key][c])
                end = time.time()
                times.append(end-start)
            break
    print(f'chunksize: {chunk_size}')
    print(np.sum(times), np.std(times))

400 Bad Request